In [2]:
import pandas as pd
import json
import requests

### 버스정류장의 경도, 위도만 있는 데이터 불러오기

In [3]:
df = pd.read_excel('서울시버스정류소좌표데이터(2020.03.06).xlsx')
df.head(3)

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN


In [4]:
bus_stops = df['정류장명']
bus_stops = pd.DataFrame(bus_stops)
bus_stops

,정류장명
0,종로2가사거리
1,창경궁.서울대학교병원
2,명륜3가.성대입구
3,종로2가.삼일교
4,혜화동로터리.여운형활동터
...,...
11275,우성아파트
11276,우성아파트
11277,조일약국
11278,성내시장


### URL형태 끝에 str으로 넣어줘야하므로 x와 y 좌표 데이터를 str으로 변경해주고 리스트로 저장

In [5]:
x_point = list(df['X좌표'])
y_point = list(df['Y좌표'])

In [6]:
x_point = map(str, x_point) # map 함수로 string으로 변환해주고
x_point = list(map(str, x_point)) # 다시 리스트로 저장

In [7]:
y_point = map(str, y_point)
y_point = list(map(str, y_point))
y_point

['37.569765125100005',
 '37.5791830159',
 '37.5826711749',
 '37.5685792736',
 '37.586242999999996',
 '37.566137',
 '37.569134999999996',
 '37.569412',
 '37.570135474000004',
 '37.570216807',
 '37.5701184824',
 '37.5702290311',
 '37.5701386788',
 '37.5702571998',
 '37.5702449756',
 '37.5703605443',
 '37.5705069968',
 '37.5706271118',
 '37.570756793',
 '37.5708806279',
 '37.5709624756',
 '37.5711272339',
 '37.572302321900004',
 '37.5719529259',
 '37.5737255179',
 '37.574243448400004',
 '37.5875502159',
 '37.5832955233',
 '37.5871907861',
 '37.585113193000005',
 '37.5878519552',
 '37.5921866176',
 '37.5923353152',
 '37.5881285775',
 '37.584966342600005',
 '37.5803257033',
 '37.580513211799996',
 '37.583091043',
 '37.5751355316',
 '37.5761834563',
 '37.5768666917',
 '37.577671386300004',
 '37.5755767894',
 '37.5724064843',
 '37.569845398000005',
 '37.570023',
 '37.5702880403',
 '37.570365',
 '37.5726577416',
 '37.5757446636',
 '37.574855345799996',
 '37.569450752',
 '37.5759017489',
 '37.5

### 카카오 API를 활용해 각 버스정류장의 행정동을 불러온다
1. api를 통해 불러온 행정동을 리스트 형태로 넣어줄 hdong 리스트를 선언
2. 위에 리스트로 만들어준 x,y 좌표를 활용할 함수 작성 (행정동을 불러와서 hdong에 append)
3. try, exept를 활용해서 api가 인식하지 못하는 좌표는 error로 기입하도록 함

In [8]:
hdong = []

def getAddress(gy,whi):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?input_coord=WGS84&output_coord=WGS84&' + 'x=' +gy+ '&y=' +whi
    headers = {"Authorization": "KakaoAK 4a9c3c0f583e534cc879947b638197c4"}
    api_test = requests.get(url,headers=headers)
    url_text = json.loads(api_test.text)
    hdong.append(url_text['documents'][1]['region_3depth_name'])

In [9]:
for i in range (0,11280): # 위 데이터프레임 길이에 맞춰서 for 문 작성
    try:
        getAddress(x_point[i], y_point[i])
    except:
        hdong.append('error')

In [10]:
len(hdong) # hdong (행정동 리스트)도 처음 데이터 프레임과 동일한 값인 것을 확인

11280

### 다른 곳에 저장되어 있던 행정동별 코드 파일을 불러온다
* 이름이 같은 동이 있으므로 이를 방지하기 위해 동별 코드 데이터와 병합 필수

In [18]:
all_dong = pd.read_excel('동별코드.xlsx', index_col = 0)
all_dong_list = list(all_dong['동']) # concat해주기

In [35]:
all_dong_list

['사직동',
 '삼청동',
 '부암동',
 '평창동',
 '무악동',
 '교남동',
 '가회동',
 '종로1.2.3.4가동',
 '종로5·6가동',
 '이화동',
 '창신1동',
 '창신2동',
 '창신3동',
 '숭인1동',
 '숭인2동',
 '청운효자동',
 '혜화동',
 '소공동',
 '회현동',
 '명동',
 '필동',
 '장충동',
 '광희동',
 '을지로동',
 '신당5동',
 '황학동',
 '중림동',
 '신당동',
 '다산동',
 '약수동',
 '청구동',
 '동화동',
 '후암동',
 '용산2가동',
 '남영동',
 '원효로2동',
 '효창동',
 '용문동',
 '이촌1동',
 '이촌2동',
 '이태원1동',
 '이태원2동',
 '서빙고동',
 '보광동',
 '청파동',
 '원효로1동',
 '한강로동',
 '한남동',
 '왕십리2동',
 '마장동',
 '사근동',
 '행당1동',
 '행당2동',
 '응봉동',
 '금호1가동',
 '금호4가동',
 '성수1가1동',
 '성수1가2동',
 '성수2가1동',
 '성수2가3동',
 '송정동',
 '용답동',
 '왕십리도선동',
 '금호2.3가동',
 '옥수동',
 '화양동',
 '군자동',
 '중곡1동',
 '중곡2동',
 '중곡3동',
 '중곡4동',
 '능동',
 '구의1동',
 '구의2동',
 '구의3동',
 '광장동',
 '자양1동',
 '자양2동',
 '자양3동',
 '자양4동',
 '회기동',
 '휘경1동',
 '휘경2동',
 '청량리동',
 '용신동',
 '제기동',
 '전농1동',
 '전농2동',
 '답십리1동',
 '답십리2동',
 '장안1동',
 '장안2동',
 '이문1동',
 '이문2동',
 '면목2동',
 '면목4동',
 '면목5동',
 '면목7동',
 '상봉1동',
 '상봉2동',
 '중화1동',
 '중화2동',
 '묵1동',
 '묵2동',
 '망우3동',
 '신내1동',
 '신내2동',
 '면목본동',
 '면목3.8동',
 '망우본동',
 '돈암1동',
 '돈암2동',
 '안

In [38]:
a = pd.DataFrame({'x': x_point, 'y': y_point, 'hdong': hdong})
a = pd.concat([a, bus_stops], axis = 1)
a.to_excel('버스정류장_행정동&정류장명.xlsx')